In [17]:
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

* **This requires plotly 4.0 or greater** *

### Dashapps to display:
1. implied volatility vs time and price using a constant underlying price

  * Calculate the implied volatility of each days options as if the underlying futures contract were  always some fixed value. For example, if the actual futures is 100, make that price 50, and then change all of the strike prices so that the strike prices are relative to the value 50.  

    1. The 100 call would be a 50 dollar call.  
    2. The 95 put would be a 45 dollar put.  
    3. The 105 call would be a 55 dollar call
   
2. Historical vs Implied Vol
3. General CSV Viewer and Grapher (graphing to come later)

In [18]:
import pandas as pd
import numpy as np
import datetime
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from dashapp import single_page_from_df as spfd
# from dashapp import dashapp2 as dashapp
import pyarrow as pa
import redis
import pdb
import pathlib


In [19]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [20]:
def get_redis_df(key):
    context = pa.default_serialization_context()
    df = context.deserialize(redis_db.get(key))
    return df

### Method to get constant volatility data from redis

In [21]:
def get_cuv_imp_from_redis():
    df_cuv_imp2 = get_redis_df('df_cuv_implied')
    df_cuv_imp2['commod'] = df_cuv_imp2.symbol.str.slice(0,2)
    df_cuv_imp2['contract_year'] = df_cuv_imp2.symbol.str.slice(-2,).astype(int) + 2000
    return df_cuv_imp2
        

In [22]:
if __name__=='__main__':
    page_title = """Show ATM Implied Volatilities
    using a
    Constant Underlying Price"""
    # *************** Create other rows ****************************
    spfd.create_dashgraph_page(
        'df_cuv_implied',get_cuv_imp_from_redis,
        app_title="Constant Underlying Vol",
        page_title=page_title,
        app_port=8814,
        run=True,
    )


2021-03-19 07:35:37,160 - root - DEBUG - df_cuv_implied entering create_dt_div
2021-03-19 07:35:37,181 - root - DEBUG - df_cuv_implied exiting create_dt_div
2021-03-19 07:35:37,278 - root - INFO - This app will run at the URL: http://127.0.0.1:8814


Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/



2021-03-19 07:35:37,280 - dashapp.dashapp2 - INFO - Dash is running on http://127.0.0.1:8814/



2021-03-19 07:35:37,286 - dashapp.dashapp2 - INFO -  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



...

_filter_query: settle_date > 20200101
_dash_table_update_paging_closure: page_current:0 page_size: 20


2021-03-19 07:36:16,159 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:36:16] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:36:16,173 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:36:16] "POST /_dash-update-component HTTP/1.1" 200 -


_filter_query: settle_date > 20210101
_dash_table_update_paging_closure: page_current:0 page_size: 20


2021-03-19 07:36:22,167 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:36:22] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:36:22,195 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:36:22] "POST /_dash-update-component HTTP/1.1" 200 -


_filter_query: settle_date > 20200101
_dash_table_update_paging_closure: page_current:0 page_size: 20


###  Historical vs Implied Vol

In [23]:
df_iv_final = pd.read_csv('./temp_folder/df_iv_final_CL.csv')

In [24]:
df_iv_final2 = df_iv_final[['settle_date','symbol','atm_iv','close_y']]
df_iv_final2 = df_iv_final2.groupby(['settle_date','symbol'],as_index=False).last()
# df_iv_final3 = df_iv_final2[['settle_date','close_y']]
#df.groupby('id')['x'].rolling(2).mean()
# df_iv_final3['hist_vol'] = df_iv_final.close_y..apply.rolling(20).std()*256**.5
# df_iv_final3 = df_iv_final3[~df_iv_final3.hist_vol.isna()]


In [25]:
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()


  sec_db


In [26]:
def get_fut(symbol):
    sql = f"select * from {futtab} where symbol='{symbol}';"
    df_fut =  pga.get_sql(sql)
    df_fut = df_fut[['settle_date','symbol','close']]
    df_fut['hist_vol'] = df_fut.close.pct_change().rolling(20,min_periods=20).std()*256**.5
    df_fut = df_fut[~df_fut.hist_vol.isna()]
    return df_fut

In [27]:
df_all_fut = None
for symbol in df_iv_final2.symbol.unique():
    df_fut = get_fut(symbol)
    if df_all_fut is None:
        df_all_fut = df_fut.copy()
    else:
        df_all_fut = df_all_fut.append(df_fut)

df_iv_final3 = df_iv_final2.merge(df_all_fut,how='inner',on=['settle_date','symbol'])
df_iv_final3.atm_iv = df_iv_final3.atm_iv.round(5)*100
df_iv_final3.hist_vol = df_iv_final3.hist_vol.round(5)*100
df_iv_final3['vol_diff'] = df_iv_final3.atm_iv - df_iv_final3.hist_vol

df_iv_final3        

,settle_date,symbol,atm_iv,close_y,close,hist_vol,vol_diff
0,20110103,CLH11,27.066,92.43,92.43,11.586,15.480
1,20110104,CLH11,28.839,90.42,90.42,14.249,14.590
2,20110105,CLH11,27.565,91.42,91.42,14.677,12.888
3,20110106,CLH11,28.640,89.75,89.75,16.138,12.502
4,20110107,CLH11,28.532,89.22,89.22,16.289,12.243
...,...,...,...,...,...,...,...
2543,20210208,CLJ21,31.108,57.83,57.83,20.446,10.662
2544,20210209,CLJ21,30.181,58.25,58.25,20.367,9.814
2545,20210210,CLJ21,29.957,58.57,58.57,19.789,10.168
2546,20210211,CLJ21,30.499,58.14,58.14,19.995,10.504


In [28]:
if __name__=='__main__':
    page_title = """Show ATM vs Historical"""
#     f = lambda:df_iv_final3
    f = spfd.dcc.Store(id='init_df_data_store',data=df_iv_final3.to_dict('records'))
    
    d = spfd.create_dashgraph_page(
        'df_iv_final3',f,
        app_title="Implied vs Historical",
        page_title=page_title,
        app_port=8814,
        run=True
    )


2021-03-19 07:37:09,227 - root - DEBUG - df_iv_final3 entering create_dt_div
2021-03-19 07:37:09,230 - root - DEBUG - df_iv_final3 exiting create_dt_div
2021-03-19 07:37:09,243 - root - INFO - This app will run at the URL: http://127.0.0.1:8814


Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/



2021-03-19 07:37:09,244 - dashapp.dashapp2 - INFO - Dash is running on http://127.0.0.1:8814/



2021-03-19 07:37:09,251 - dashapp.dashapp2 - INFO -  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



...

2021-03-19 07:38:25,495 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:38:25] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:38:25,516 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:38:25] "POST /_dash-update-component HTTP/1.1" 200 -


filter contains: CLZ19
_dash_table_update_paging_closure: page_current:0 page_size: 20


2021-03-19 07:39:05,710 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:39:05] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:39:05,728 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:39:05] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:39:10,305 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:39:10] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:39:23,315 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:39:23] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:41:03,831 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:41:03] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:41:06,826 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:41:06] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:41:49,182 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:41:49] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:41:52,525 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:41:52] "POST /_dash-update-component 

filter contains: CLX19
_dash_table_update_paging_closure: page_current:0 page_size: 20


2021-03-19 07:42:44,643 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:42:44] "POST /_dash-update-component HTTP/1.1" 200 -


### App with CSV Uploader


In [29]:
import importlib
importlib.reload(spfd)
importlib.reload(spfd.dashapp)
if __name__=='__main__':
    logger = spfd.dashapp.init_root_logger()    
    spfd.graph_from_csv_page(main_id='myid',app_port = 8814,app_title="csv_grapher",
        logger=logger)

2021-03-19 07:45:10,511 - root - DEBUG - myid_dtdf entering create_dt_div
2021-03-19 07:45:10,513 - root - DEBUG - myid_dtdf exiting create_dt_div
2021-03-19 07:45:10,524 - root - INFO - This app will run at the URL: http://127.0.0.1:8814


Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/

Dash is running on http://127.0.0.1:8814/



2021-03-19 07:45:10,526 - dashapp.dashapp2 - INFO - Dash is running on http://127.0.0.1:8814/



2021-03-19 07:45:10,536 - dashapp.dashapp2 - INFO -  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



...

2021-03-19 07:46:34,462 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:46:34] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:46:35,555 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:46:35] "POST /_dash-update-component HTTP/1.1" 200 -


_dash_table_update_paging_closure: page_current:0 page_size: 100


2021-03-19 07:46:39,715 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:46:39] "POST /_dash-update-component HTTP/1.1" 200 -
2021-03-19 07:46:39,869 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:46:39] "POST /_dash-update-component HTTP/1.1" 200 -


_dash_table_update_paging_closure: page_current:1 page_size: 100


2021-03-19 07:47:47,498 - werkzeug - INFO - 127.0.0.1 - - [19/Mar/2021 07:47:47] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
# ?spfd.html.Button

## END